# Linear Algebra
(most of the material for this notebook has been copied from [1](https://github.com/xorJane/Introduction_to_Julia_tutorials/blob/master/11.Basic_linear_algebra.ipynb) and [2](https://github.com/xorJane/Introduction_to_Julia_tutorials/blob/master/12.Factorizations_and_other_fun.ipynb))
Topics:
- Basic linear algebra operations
- `LinearAlgebra.jl` package
  - Factorization
  - Special matrix structures
  - Generic linear algebra


## Basic linear algebra
First let's define a random matrix

In [ ]:
A = rand(1:4,3,3)

Define a vector of ones

In [ ]:
x = fill(1.0, 3)

Notice that `A` has type `Array{Int64,2}` but `x` has type `Array{Float64,1}`. Julia defines the aliases `Vector{Type}=Array{Type,1}` and `Matrix{Type}=Array{Type,2}`.

Many of the basic operations are the same as in other languages
### Multiplication

In [ ]:
b = A*x

### Transposition
As in other languages `A'` is the conjugate transpose, or adjoint

In [ ]:
A'

and we can get the transpose with

In [ ]:
transpose(A)

### Transposed multiplication
Julia allows us to write this without `*`


In [ ]:
A'A

### Solving linear systems

The problem $Ax = b$ for *square* $A$ is solved by the `\` function.

In [ ]:
A\b

`A\b` gives us the least squares solution if we have an overdetermined linear system (a "tall" matrix)

In [ ]:
Atall = rand(3, 2)

In [ ]:
Atall\b

and the *minimum norm least squares solution* if we have a rank-deficient least squares problem

In [ ]:
v = rand(3)
rankdef = hcat(v, v)

In [ ]:
rankdef\b

Julia also gives us the minimum norm solution when we have an underdetermined solution (a "short" matrix)

In [ ]:
bshort = rand(2)
Ashort = rand(2, 3)

In [ ]:
Ashort\bshort

## `LinearAlgebral.jl` package
Most of linear algebra operators are included by default in Julia's Base package. However, more advanced functionality is possible with the standard package `LinearAlgebra.jl`. Let's see it in action:


In [ ]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x
     


### Matrix factorization:
#### LU factorization
We can perform an *LU* factorization:
$$PA = LU\, ,$$
where $P$ is a permutation matrix,$L$ is lower-triangular unit diagonal and $U$ is upper-trianguler.
Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [ ]:
Alu = lu(A)

In [ ]:
typeof(Alu)

The different parts of the factorization can be extracted by accessing their special properties

In [ ]:
Alu.P

In [ ]:
Alu.L

In [ ]:
Alu.U

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [ ]:
A\b

In [ ]:
Alu\b

Similarly, we can calculate the determinant of $A$ using either `A` or the factorization object

In [ ]:
det(A) ≈ det(Alu)

#### QR factorizations

In Julia we can perform a QR factorization

$$A=QR$$

where $Q$ is unitary/orthogonal and $R$ is upper triangular.

In [ ]:
Aqr = qr(A)

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [ ]:
Aqr.Q

In [ ]:
Aqr.R

#### Eigendecompositions
`LinearAlgebra.jl` offers many different decomposition, including  eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions that are all stored in `Factorization` types.

The eigendecomposition can be computed:

In [ ]:
Asym = A + A'
AsymEig = eigen(Asym)

The values and the vectors can be extracted from the Eigen type by special indexing

In [ ]:
AsymEig.values

In [ ]:
AsymEig.vectors

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that 
$$A^{-1} = (V \Lambda V^{-1})^{-1} = V \Lambda^{-1} V^{-1}$$

In [ ]:
inv(AsymEig)*Asym

### Special matrix structures
Matrix structure is very important in linear algebra. To see how important it is, let's work with a larger linear system

In [ ]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [ ]:
Asym = A + A'
issymmetric(Asym)

but sometimes floating point error might get in the way.

In [ ]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

In [ ]:
issymmetric(Asym_noisy)

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [ ]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark eigvals(Asym)

In [ ]:
@benchmark eigvals(Asym_noisy)

In [ ]:
@benchmark eigvals(Asym_explicit)

In this example, using `Symmetric()` on `Asym_noisy` made our calculations about 5x more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [ ]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@btime eigmax(A)

###  Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers or [quaternions](https://en.wikipedia.org/wiki/Quaternion) and [octonions](https://en.wikipedia.org/wiki/Octonion)!

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInts`.

In [ ]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

In [ ]:
x = fill(1, 3)
b = Arational*x

In [ ]:
Arational\b

In [ ]:
lu(Arational)

#### Generic matrix factorization for octonions!
(Credit to Seth Axen on Slack)

 It's very impressive that matrix factorization works for quaternions as well as octonions (with neither commutative nor associative multiplication)!!

In [ ]:
using Octonions
x = rand(OctonionF64, 5, 5)
Q , R = qr(x);

In [ ]:
(Q * R ≈ x) && (Q'Q ≈ I) && istriu(R)

In [ ]:
L, U = lu(x, NoPivot());
(L * U ≈ x) && istril(L) && (Diagonal(L) ≈ I) && istriu(U)